### load modules

In [1]:
%matplotlib inline
import numpy as np
import os, glob
import matplotlib.pyplot as plt
from netCDF4 import Dataset

In [2]:
import timeit
from pandas import to_datetime
import wrf, xarray

### define paths

In [3]:
dataPath = '/glade/scratch/domingom/Cheyenne/XPIA_mesoLES/SIMULS/'
outPath  = '/glade/scratch/doubrawa/postProcessing/'

### define some parameters

In [4]:
n_processors = 1800
variables    = ['wa','T','XLAT','XLONG']
domainId     = 3
height       = 100.0
prefix       = "LES_25m" if domainId==4 else "LES_100m"

### allocate space for U/V in the shape (time, south_north, east_west)

In [5]:
nt_per_file        = 180 if domainId==4 else 30
nt_total           = nt_per_file 

n_east_west        = 1200
n_east_west_stag   = n_east_west + 1
n_south_north      = 1200
n_south_north_stag = n_south_north + 1

### loop processors and accumulate data

In [6]:
day           = 27
hour          = 15

#
# For each day/hour, allocate space for the variables and set the startTime to None
#
dateStart     = None

data = {}
for varName in variables:
    if ( (varName!="XLAT") & (varName!="XLONG") ):
        data[varName] = np.zeros((nt_total, n_south_north, n_east_west ))      
    else:
        data[varName] = np.zeros((n_south_north, n_east_west ))      

In [7]:
tic=timeit.default_timer()        
   
for half_hour_idx in [1]:
    
    new_half_hour = 1
    
    #
    # Find out the expected indices along the time dimension
    #
    time_start    = 0 
    time_end      = time_start+nt_per_file    
    
    for processor in range(n_processors):
        wrfoutPath    = glob.glob(os.path.join(dataPath,
                               'WRF_mesoLES_4dom_RAP_2015_03_{0}_mesoLES'.format(day),
                               'HOUR_{0}_{1}'.format(hour,half_hour_idx),
                               'wrfout_d0{0}_2015-03-*_{1:04d}'.format(domainId,processor)))
        print os.path.split(wrfoutPath[0])[-1]

        #
        # Read in the data, and grab U/V
        #
        ncfile = Dataset(wrfoutPath[0],'r')
        
        #
        # Get start/end indices
        #
        we_0 = getattr(ncfile,'WEST-EAST_PATCH_START_UNSTAG') - 1        
        we_1 = getattr(ncfile,'WEST-EAST_PATCH_END_UNSTAG')                
        
        sn_0 = getattr(ncfile,'SOUTH-NORTH_PATCH_START_UNSTAG') - 1       
        sn_1 = getattr(ncfile,'SOUTH-NORTH_PATCH_END_UNSTAG')                                
        
        #
        # Make a list of datetimes
        #
        if new_half_hour:
            datetimes_now = [to_datetime(''.join(x.tolist()),format='%Y-%m-%d_%H:%M:%S') for x in ncfile.variables['Times'][:]]
            new_half_hour = 0
        
        #
        # get all variables (including those which are only needed for vertical interpolation)
        #
        
        z    = wrf.getvar(ncfile, "z", timeidx=wrf.ALL_TIMES)
        ter  = wrf.getvar(ncfile, "ter", timeidx=wrf.ALL_TIMES)      
        zref = z-ter 
        
        for varName in variables:
            
            if ( (varName!="XLAT") & (varName!="XLONG") ):

                var     = wrf.getvar(ncfile, varName, timeidx=wrf.ALL_TIMES)
                var_int = wrf.interplevel(var, zref, height, meta=True)
                data[varName][time_start:time_end, sn_0:sn_1, we_0:we_1] = var_int
        
            else:
                
                var_int = wrf.getvar(ncfile, varName)
                data[varName][sn_0:sn_1, we_0:we_1] = var_int

            
dateEnd = ''.join(ncfile.variables['Times'][:][-1].tolist()) 
toc=timeit.default_timer()

wrfout_d03_2015-03-28_15:01:00_0000
wrfout_d03_2015-03-28_15:01:00_0001
wrfout_d03_2015-03-28_15:01:00_0002
wrfout_d03_2015-03-28_15:01:00_0003
wrfout_d03_2015-03-28_15:01:00_0004
wrfout_d03_2015-03-28_15:01:00_0005
wrfout_d03_2015-03-28_15:01:00_0006
wrfout_d03_2015-03-28_15:01:00_0007
wrfout_d03_2015-03-28_15:01:00_0008
wrfout_d03_2015-03-28_15:01:00_0009
wrfout_d03_2015-03-28_15:01:00_0010
wrfout_d03_2015-03-28_15:01:00_0011
wrfout_d03_2015-03-28_15:01:00_0012
wrfout_d03_2015-03-28_15:01:00_0013
wrfout_d03_2015-03-28_15:01:00_0014
wrfout_d03_2015-03-28_15:01:00_0015
wrfout_d03_2015-03-28_15:01:00_0016
wrfout_d03_2015-03-28_15:01:00_0017
wrfout_d03_2015-03-28_15:01:00_0018
wrfout_d03_2015-03-28_15:01:00_0019
wrfout_d03_2015-03-28_15:01:00_0020
wrfout_d03_2015-03-28_15:01:00_0021
wrfout_d03_2015-03-28_15:01:00_0022
wrfout_d03_2015-03-28_15:01:00_0023
wrfout_d03_2015-03-28_15:01:00_0024
wrfout_d03_2015-03-28_15:01:00_0025
wrfout_d03_2015-03-28_15:01:00_0026
wrfout_d03_2015-03-28_15:01:

wrfout_d03_2015-03-28_15:01:00_0228
wrfout_d03_2015-03-28_15:01:00_0229
wrfout_d03_2015-03-28_15:01:00_0230
wrfout_d03_2015-03-28_15:01:00_0231
wrfout_d03_2015-03-28_15:01:00_0232
wrfout_d03_2015-03-28_15:01:00_0233
wrfout_d03_2015-03-28_15:01:00_0234
wrfout_d03_2015-03-28_15:01:00_0235
wrfout_d03_2015-03-28_15:01:00_0236
wrfout_d03_2015-03-28_15:01:00_0237
wrfout_d03_2015-03-28_15:01:00_0238
wrfout_d03_2015-03-28_15:01:00_0239
wrfout_d03_2015-03-28_15:01:00_0240
wrfout_d03_2015-03-28_15:01:00_0241
wrfout_d03_2015-03-28_15:01:00_0242
wrfout_d03_2015-03-28_15:01:00_0243
wrfout_d03_2015-03-28_15:01:00_0244
wrfout_d03_2015-03-28_15:01:00_0245
wrfout_d03_2015-03-28_15:01:00_0246
wrfout_d03_2015-03-28_15:01:00_0247
wrfout_d03_2015-03-28_15:01:00_0248
wrfout_d03_2015-03-28_15:01:00_0249
wrfout_d03_2015-03-28_15:01:00_0250
wrfout_d03_2015-03-28_15:01:00_0251
wrfout_d03_2015-03-28_15:01:00_0252
wrfout_d03_2015-03-28_15:01:00_0253
wrfout_d03_2015-03-28_15:01:00_0254
wrfout_d03_2015-03-28_15:01:

wrfout_d03_2015-03-28_15:01:00_0456
wrfout_d03_2015-03-28_15:01:00_0457
wrfout_d03_2015-03-28_15:01:00_0458
wrfout_d03_2015-03-28_15:01:00_0459
wrfout_d03_2015-03-28_15:01:00_0460
wrfout_d03_2015-03-28_15:01:00_0461
wrfout_d03_2015-03-28_15:01:00_0462
wrfout_d03_2015-03-28_15:01:00_0463
wrfout_d03_2015-03-28_15:01:00_0464
wrfout_d03_2015-03-28_15:01:00_0465
wrfout_d03_2015-03-28_15:01:00_0466
wrfout_d03_2015-03-28_15:01:00_0467
wrfout_d03_2015-03-28_15:01:00_0468
wrfout_d03_2015-03-28_15:01:00_0469
wrfout_d03_2015-03-28_15:01:00_0470
wrfout_d03_2015-03-28_15:01:00_0471
wrfout_d03_2015-03-28_15:01:00_0472
wrfout_d03_2015-03-28_15:01:00_0473
wrfout_d03_2015-03-28_15:01:00_0474
wrfout_d03_2015-03-28_15:01:00_0475
wrfout_d03_2015-03-28_15:01:00_0476
wrfout_d03_2015-03-28_15:01:00_0477
wrfout_d03_2015-03-28_15:01:00_0478
wrfout_d03_2015-03-28_15:01:00_0479
wrfout_d03_2015-03-28_15:01:00_0480
wrfout_d03_2015-03-28_15:01:00_0481
wrfout_d03_2015-03-28_15:01:00_0482
wrfout_d03_2015-03-28_15:01:

wrfout_d03_2015-03-28_15:01:00_0684
wrfout_d03_2015-03-28_15:01:00_0685
wrfout_d03_2015-03-28_15:01:00_0686
wrfout_d03_2015-03-28_15:01:00_0687
wrfout_d03_2015-03-28_15:01:00_0688
wrfout_d03_2015-03-28_15:01:00_0689
wrfout_d03_2015-03-28_15:01:00_0690
wrfout_d03_2015-03-28_15:01:00_0691
wrfout_d03_2015-03-28_15:01:00_0692
wrfout_d03_2015-03-28_15:01:00_0693
wrfout_d03_2015-03-28_15:01:00_0694
wrfout_d03_2015-03-28_15:01:00_0695
wrfout_d03_2015-03-28_15:01:00_0696
wrfout_d03_2015-03-28_15:01:00_0697
wrfout_d03_2015-03-28_15:01:00_0698
wrfout_d03_2015-03-28_15:01:00_0699
wrfout_d03_2015-03-28_15:01:00_0700
wrfout_d03_2015-03-28_15:01:00_0701
wrfout_d03_2015-03-28_15:01:00_0702
wrfout_d03_2015-03-28_15:01:00_0703
wrfout_d03_2015-03-28_15:01:00_0704
wrfout_d03_2015-03-28_15:01:00_0705
wrfout_d03_2015-03-28_15:01:00_0706
wrfout_d03_2015-03-28_15:01:00_0707
wrfout_d03_2015-03-28_15:01:00_0708
wrfout_d03_2015-03-28_15:01:00_0709
wrfout_d03_2015-03-28_15:01:00_0710
wrfout_d03_2015-03-28_15:01:

wrfout_d03_2015-03-28_15:01:00_0912
wrfout_d03_2015-03-28_15:01:00_0913
wrfout_d03_2015-03-28_15:01:00_0914
wrfout_d03_2015-03-28_15:01:00_0915
wrfout_d03_2015-03-28_15:01:00_0916
wrfout_d03_2015-03-28_15:01:00_0917
wrfout_d03_2015-03-28_15:01:00_0918
wrfout_d03_2015-03-28_15:01:00_0919
wrfout_d03_2015-03-28_15:01:00_0920
wrfout_d03_2015-03-28_15:01:00_0921
wrfout_d03_2015-03-28_15:01:00_0922
wrfout_d03_2015-03-28_15:01:00_0923
wrfout_d03_2015-03-28_15:01:00_0924
wrfout_d03_2015-03-28_15:01:00_0925
wrfout_d03_2015-03-28_15:01:00_0926
wrfout_d03_2015-03-28_15:01:00_0927
wrfout_d03_2015-03-28_15:01:00_0928
wrfout_d03_2015-03-28_15:01:00_0929
wrfout_d03_2015-03-28_15:01:00_0930
wrfout_d03_2015-03-28_15:01:00_0931
wrfout_d03_2015-03-28_15:01:00_0932
wrfout_d03_2015-03-28_15:01:00_0933
wrfout_d03_2015-03-28_15:01:00_0934
wrfout_d03_2015-03-28_15:01:00_0935
wrfout_d03_2015-03-28_15:01:00_0936
wrfout_d03_2015-03-28_15:01:00_0937
wrfout_d03_2015-03-28_15:01:00_0938
wrfout_d03_2015-03-28_15:01:

wrfout_d03_2015-03-28_15:01:00_1140
wrfout_d03_2015-03-28_15:01:00_1141
wrfout_d03_2015-03-28_15:01:00_1142
wrfout_d03_2015-03-28_15:01:00_1143
wrfout_d03_2015-03-28_15:01:00_1144
wrfout_d03_2015-03-28_15:01:00_1145
wrfout_d03_2015-03-28_15:01:00_1146
wrfout_d03_2015-03-28_15:01:00_1147
wrfout_d03_2015-03-28_15:01:00_1148
wrfout_d03_2015-03-28_15:01:00_1149
wrfout_d03_2015-03-28_15:01:00_1150
wrfout_d03_2015-03-28_15:01:00_1151
wrfout_d03_2015-03-28_15:01:00_1152
wrfout_d03_2015-03-28_15:01:00_1153
wrfout_d03_2015-03-28_15:01:00_1154
wrfout_d03_2015-03-28_15:01:00_1155
wrfout_d03_2015-03-28_15:01:00_1156
wrfout_d03_2015-03-28_15:01:00_1157
wrfout_d03_2015-03-28_15:01:00_1158
wrfout_d03_2015-03-28_15:01:00_1159
wrfout_d03_2015-03-28_15:01:00_1160
wrfout_d03_2015-03-28_15:01:00_1161
wrfout_d03_2015-03-28_15:01:00_1162
wrfout_d03_2015-03-28_15:01:00_1163
wrfout_d03_2015-03-28_15:01:00_1164
wrfout_d03_2015-03-28_15:01:00_1165
wrfout_d03_2015-03-28_15:01:00_1166
wrfout_d03_2015-03-28_15:01:

wrfout_d03_2015-03-28_15:01:00_1368
wrfout_d03_2015-03-28_15:01:00_1369
wrfout_d03_2015-03-28_15:01:00_1370
wrfout_d03_2015-03-28_15:01:00_1371
wrfout_d03_2015-03-28_15:01:00_1372
wrfout_d03_2015-03-28_15:01:00_1373
wrfout_d03_2015-03-28_15:01:00_1374
wrfout_d03_2015-03-28_15:01:00_1375
wrfout_d03_2015-03-28_15:01:00_1376
wrfout_d03_2015-03-28_15:01:00_1377
wrfout_d03_2015-03-28_15:01:00_1378
wrfout_d03_2015-03-28_15:01:00_1379
wrfout_d03_2015-03-28_15:01:00_1380
wrfout_d03_2015-03-28_15:01:00_1381
wrfout_d03_2015-03-28_15:01:00_1382
wrfout_d03_2015-03-28_15:01:00_1383
wrfout_d03_2015-03-28_15:01:00_1384
wrfout_d03_2015-03-28_15:01:00_1385
wrfout_d03_2015-03-28_15:01:00_1386
wrfout_d03_2015-03-28_15:01:00_1387
wrfout_d03_2015-03-28_15:01:00_1388
wrfout_d03_2015-03-28_15:01:00_1389
wrfout_d03_2015-03-28_15:01:00_1390
wrfout_d03_2015-03-28_15:01:00_1391
wrfout_d03_2015-03-28_15:01:00_1392
wrfout_d03_2015-03-28_15:01:00_1393
wrfout_d03_2015-03-28_15:01:00_1394
wrfout_d03_2015-03-28_15:01:

wrfout_d03_2015-03-28_15:01:00_1596
wrfout_d03_2015-03-28_15:01:00_1597
wrfout_d03_2015-03-28_15:01:00_1598
wrfout_d03_2015-03-28_15:01:00_1599
wrfout_d03_2015-03-28_15:01:00_1600
wrfout_d03_2015-03-28_15:01:00_1601
wrfout_d03_2015-03-28_15:01:00_1602
wrfout_d03_2015-03-28_15:01:00_1603
wrfout_d03_2015-03-28_15:01:00_1604
wrfout_d03_2015-03-28_15:01:00_1605
wrfout_d03_2015-03-28_15:01:00_1606
wrfout_d03_2015-03-28_15:01:00_1607
wrfout_d03_2015-03-28_15:01:00_1608
wrfout_d03_2015-03-28_15:01:00_1609
wrfout_d03_2015-03-28_15:01:00_1610
wrfout_d03_2015-03-28_15:01:00_1611
wrfout_d03_2015-03-28_15:01:00_1612
wrfout_d03_2015-03-28_15:01:00_1613
wrfout_d03_2015-03-28_15:01:00_1614
wrfout_d03_2015-03-28_15:01:00_1615
wrfout_d03_2015-03-28_15:01:00_1616
wrfout_d03_2015-03-28_15:01:00_1617
wrfout_d03_2015-03-28_15:01:00_1618
wrfout_d03_2015-03-28_15:01:00_1619
wrfout_d03_2015-03-28_15:01:00_1620
wrfout_d03_2015-03-28_15:01:00_1621
wrfout_d03_2015-03-28_15:01:00_1622
wrfout_d03_2015-03-28_15:01:

In [8]:
print(toc-tic)

1360.2216959


# make the data self-aware

In [9]:
xlat = xarray.DataArray(data['XLAT'], 
                 coords={"south_north":range(1200),"west_east":range(1200)}, 
                 dims=("south_north","west_east"), 
                 name="2-d latitude", 
                 attrs={"unit":"deg","stagger":""})

xlong = xarray.DataArray(data['XLONG'], 
                 coords={"south_north":range(1200),"west_east":range(1200)}, 
                 dims=("south_north","west_east"), 
                 name="2-d longitude", 
                 attrs={"unit":"deg","stagger":""})

In [10]:
w = xarray.DataArray(data['wa'], 
                 coords={"time":datetimes_now,"south_north":range(1200),"west_east":range(1200)}, 
                 dims=("time","south_north","west_east"), 
                 name="w", 
                 attrs={"unit":"m s-1","stagger":"","height [m]":height})
w["xlat"] = xlat
w["xlong"] = xlong

In [11]:
theta = xarray.DataArray(data['T'], 
                 coords={"time":datetimes_now,"south_north":range(1200),"west_east":range(1200)}, 
                 dims=("time","south_north","west_east"), 
                 name="potential temperature", 
                 attrs={"unit":"K","stagger":"","height [m]":height})
theta = theta + 300
theta["xlat"] = xlat
theta["xlong"] = xlong

### save

In [12]:
dataset = xarray.Dataset(data_vars={'w':w,'theta':theta})

In [13]:
fName = "WRF_{0}_{1:%Y-%m-%d}_{1:%H:%M}_{2:%H:%M}.nc".format(prefix,to_datetime(theta['time'].data[0]),to_datetime(theta['time'].data[-1]))
fPath = os.path.join(outPath,fName)
print "Saving : {0}".format(fName)
dataset.to_netcdf(fPath)

Saving : WRF_LES_100m_2015-03-28_15:01_15:30.nc
